In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv("BaseData.csv")
data.head()
data["month_num"] = data["month_num"].astype(int)
data["day"] = data["day"].astype(int)

data.drop(["Unnamed: 12"], axis=1)
data.head()






,maxsal,minsal,intern_num,city,scale,c_desc,month_num,day,c_tag,name,industry,c_name,Unnamed: 12
0,100,50,62,北京,2000人以上,搜狐是中国领先的互联网媒体、搜索、在线游戏集团，拥有搜狐、畅游和搜狗三家美国上市公司。,3,5,"['良好团队氛围', '有大牛', '丰厚实习佣金', '帅哥美女多', '快速成长', '...",新媒体运维实习生（线上）,互联网/游戏/软件,搜狐,NaN
1,150,100,5,深圳,15-50人,致力于为中国高端家庭提供一站式国际教育服务,4,5,"['工作氛围极好', '领导nice', '发展速度快', '专业提升巨大', '专注教育'...",人事实习生,教育/培训,海岸线国际教育,NaN
2,300,250,8,上海,15-50人,致力于提供针对非结构化数据存储、标注、模型训练和管理预测的一站式AI服务平台产品,4,5,"['弹性工作', '技术大牛', '高成长平台', 'AI技术平台']",测试实习生,互联网/游戏/软件,隐木科技,NaN
3,100,100,3,上海,15-50人,由上海万科房地产有限公司全资成立的子公司,4,4,['氛围nice'],国际学校品牌宣传实习生,教育/培训,万科教育,NaN
4,150,100,176,广州,2000人以上,以视频内容，连接你我，丰富生活,3,4,"['工作环境超棒', '节日大礼包', '餐补']",营收产品运营实习生,互联网/游戏/软件,欢聚集团,NaN


# Do one-hot on company scale

In [4]:
scale = data["scale"].to_numpy()

def map(string, vector):
    res = ["0" for i in range(len(vector))]
    for i in range(len(vector)):
        if isinstance(vector[i],tuple):
            for sub_label in vector[i]:
                if sub_label in string:
                    res[i] = "1"
        elif vector[i] in string:
            res[i] = "1"
            
    return res

scale_vector = ["2000人以上",("500-2000人","500—2000人"),("150-500人","50-100人","50-150人","50-200人"),("15-50人","少于50人"),"未知规模"]

mapped_scale = []
for i in scale:
    mapped = map(i,scale_vector)
    mapped_scale.append(mapped)
    

matrix = np.array(mapped_scale)

scale_df = pd.DataFrame(data=matrix, columns=["2000人以上","500-2000人","50-500人","少于50人","未知规模"])
scale_df.head()

,2000人以上,500-2000人,50-500人,少于50人,未知规模
0,1,0,0,0,0
1,0,0,0,1,0
2,0,0,0,1,0
3,0,0,0,1,0
4,1,0,0,0,0


# Concatenate Text Columns 

In [10]:
c_desc = data["c_desc"]
c_tag = data["c_tag"]
name = data["name"]

text_df = pd.DataFrame(c_desc+ c_tag + name, columns=["text"])
text_df.head()

,text
0,搜狐是中国领先的互联网媒体、搜索、在线游戏集团，拥有搜狐、畅游和搜狗三家美国上市公司。['良...
1,"致力于为中国高端家庭提供一站式国际教育服务['工作氛围极好', '领导nice', '发展速..."
2,致力于提供针对非结构化数据存储、标注、模型训练和管理预测的一站式AI服务平台产品['弹性工作...
3,由上海万科房地产有限公司全资成立的子公司['氛围nice']国际学校品牌宣传实习生
4,"以视频内容，连接你我，丰富生活['工作环境超棒', '节日大礼包', '餐补']营收产品运营实习生"


# Do TF-IDF on Text

In [29]:
import jieba
import re
from sklearn.feature_extraction.text import TfidfVectorizer

#Split words
des = text_df["text"].to_list()

cutted_des = [list(jieba.cut(re.sub(r"[0-9\s+\.\!\/_,$%^*()?;；:-【】+\"\']+|[+——！，;:。？、~@#￥%……&*（）]+", " ", sentence))) for sentence in des]
document = [" ".join(cutted) for cutted in cutted_des]

#Load stop words
with open("cn_stopwords.txt","r",encoding="utf-8") as fp:
    word_list = fp.read().split("\n")
    
extra_wordlist = ["一代","一名","一所","一批","一支","一款","一直","一种","一级","一群"]
word_list += extra_wordlist

tfidf_model = TfidfVectorizer(stop_words=word_list, max_df = 0.005, min_df=0.0001).fit(document)

a = tfidf_model.transform(document).todense()
volcab = tfidf_model.vocabulary_
print(volcab)

name_header = [0]*len(volcab)
for name, val in volcab.items():
    name_header[val] = name

name_df=pd.DataFrame(data=a, columns = name_header)
name_df.head()

c:\users\yz391\appdata\local\programs\python\python37\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['若果'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{'搜狐': 2410, '搜索': 2412, '畅游': 3325, '搜狗': 2411, '三家': 50, '佣金': 565, '家庭': 1768, '顶级': 4630, '结构化': 3679, '存储': 1658, '标注': 2822, '模型': 2873, '训练': 4075, '预测': 4639, '万科': 40, '学校': 1674, '宣传': 1754, '营收': 3950, '医疗机构': 999, '医学': 990, '新加坡': 2549, '猎头公司': 3194, '北美': 980, '中科院': 224, '中科': 223, '创星': 857, '茶水': 3925, '身体': 4272, '吃喝玩乐': 1175, '小米': 1824, '清洁': 3073, '类产品': 3591, '爆款': 3151, '米家': 3590, '扫地机': 2285, '石头': 3409, '电池': 3309, '八方': 697, '锦程': 4500, '雇前': 4572, '调查': 4139, '极为': 2799, '很帅': 2096, '费用': 4199, '欢乐': 2882, '婚假': 1650, '产假': 384, '规定': 4040, '双休日': 1119, '浩鲸云': 3019, '计算': 4056, '科技股份': 3483, '阿里巴巴': 4533, '中兴通讯': 183, '业内': 132, '认可度': 4070, '政治': 2455, '托福': 2276, '南京': 1060, '摩亚': 2422, '新西兰': 2567, '卢卡': 1081, '蜂业': 3979, '授权': 2372, '绿色': 3709, '技能': 2308, '五星': 349, '区属': 987, '南沙': 1066, '属下': 1858, '终生': 3659, '可转': 1169, '编制': 3712, '香港联交所': 4719, '集团公司': 4562, '直营': 3375, '超市': 4230, '主营业务': 273, '囊括': 1316, '便利店': 573, '商业广场': 1274, '多家': 1502, '中餐

,一九八九年,一亿人,一件,一份,一卡通,一只,一同,一块,一天,一带,...,麻醉学,黄埔军校,黄金珠宝,黑快,鼓励,龙光,龙头,龙头企业,龙川,龙湖
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Concatenate Dataframes**

In [48]:
frames = [data,scale_df, name_df]
result = pd.concat(frames, axis=1)
result.head()

,maxsal,minsal,intern_num,city,scale,c_desc,month_num,day,c_tag,name,...,项目,项目管理,项目经理,顾问,领域,风控,高中,高中地理,高中数学,高中物理
0,100,50,62,北京,2000人以上,搜狐是中国领先的互联网媒体、搜索、在线游戏集团，拥有搜狐、畅游和搜狗三家美国上市公司。,3,5,"['良好团队氛围', '有大牛', '丰厚实习佣金', '帅哥美女多', '快速成长', '...",新媒体运维实习生（线上）,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,150,100,5,深圳,15-50人,致力于为中国高端家庭提供一站式国际教育服务,4,5,"['工作氛围极好', '领导nice', '发展速度快', '专业提升巨大', '专注教育'...",人事实习生,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,300,250,8,上海,15-50人,致力于提供针对非结构化数据存储、标注、模型训练和管理预测的一站式AI服务平台产品,4,5,"['弹性工作', '技术大牛', '高成长平台', 'AI技术平台']",测试实习生,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100,100,3,上海,15-50人,由上海万科房地产有限公司全资成立的子公司,4,4,['氛围nice'],国际学校品牌宣传实习生,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,150,100,176,广州,2000人以上,以视频内容，连接你我，丰富生活,3,4,"['工作环境超棒', '节日大礼包', '餐补']",营收产品运营实习生,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
result.to_csv("result.csv")

In [50]:
df = pd.read_csv("result.csv")
df.head()

c:\users\yz391\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,maxsal,minsal,intern_num,city,scale,c_desc,month_num,day,c_tag,...,项目,项目管理,项目经理,顾问,领域,风控,高中,高中地理,高中数学,高中物理
0,0,100,50,62,北京,2000人以上,搜狐是中国领先的互联网媒体、搜索、在线游戏集团，拥有搜狐、畅游和搜狗三家美国上市公司。,3,5,"['良好团队氛围', '有大牛', '丰厚实习佣金', '帅哥美女多', '快速成长', '...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,150,100,5,深圳,15-50人,致力于为中国高端家庭提供一站式国际教育服务,4,5,"['工作氛围极好', '领导nice', '发展速度快', '专业提升巨大', '专注教育'...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,300,250,8,上海,15-50人,致力于提供针对非结构化数据存储、标注、模型训练和管理预测的一站式AI服务平台产品,4,5,"['弹性工作', '技术大牛', '高成长平台', 'AI技术平台']",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,100,100,3,上海,15-50人,由上海万科房地产有限公司全资成立的子公司,4,4,['氛围nice'],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,150,100,176,广州,2000人以上,以视频内容，连接你我，丰富生活,3,4,"['工作环境超棒', '节日大礼包', '餐补']",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


0       1
1       0
2       0
3       0
4       1
       ..
4994    1
4995    0
4996    0
4997    0
4998    0
Name: 2000人以上, Length: 4999, dtype: int64